In [7]:
import pandas as pd
import re
import json
import sys
from fuzzywuzzy import fuzz

# ==============================================================================
# 1. CONFIGURAÇÃO DO PROJETO E ARQUIVOS DE ENTRADA
# ==============================================================================

# Caminho do arquivo de dados (Excel)
EXCEL_FILE = r'C:\Users\ane.freitas\Downloads\Case - Wpp (1).xlsx' 
# Nome da aba/planilha dentro do Excel que será lida
SHEET_NAME = 'result' 
# Nome da coluna que contém o texto de entrada para todas as análises
COLUNA_ORIGEM = 'input' 

# Dicionários (Regras de Negócio) para Classificação Textual
DIC_OBJETIVO_FILE = 'dicionario_objetivo.json'
DIC_RESGATE_FILE = 'dicionario_resgate.json'
# ==============================================================================
# 2. FUNÇÕES BASE DE SUPORTE (CARREGAMENTO E CLASSIFICAÇÃO)
# ==============================================================================

def carregar_dicionario(file_path):
    """Carrega as Regras de Classificação (dicionários JSON)."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        mapping = {}
        for categoria, frases_lista in data.items():
            for frase in frases_lista:
                mapping[frase.lower()] = categoria 
        
        print(f"Dicionário '{file_path}' carregado com sucesso.")
        return mapping
            
    except FileNotFoundError:
        print(f"\nERRO FATAL: O arquivo do dicionário '{file_path}' não foi encontrado.")
        sys.exit(1)

# Carrega os dicionários para as análises de Objetivo e Resgate
MAPA_OBJETIVOS = carregar_dicionario(DIC_OBJETIVO_FILE)
MAPA_RESGATE = carregar_dicionario(DIC_RESGATE_FILE)

def classificar_valor_aporte(texto):
    """
    Classificação de Aporte: Extrai o valor (Regex com ajuste para milhar BR) 
    e aplica as regras de Aporte Baixo/Médio/Alto.
    """
    if pd.isna(texto) or not isinstance(texto, (str, int, float)) or str(texto).strip() == "":
        return None, 'NÃO SE APLICA'
    
    texto_limpo = str(texto).lower().strip().replace('r$', '').replace('$', '').replace('reais', '')
    
    if len(texto_limpo) > 50 or "http" in texto_limpo or "media_" in texto_limpo:
        return None, 'NÃO SE APLICA'

    # Lógica de Extração (Regex)
    match = re.search(r'(\d[\d\.\,]*)(\s*mil|\s*k)?', texto_limpo)
    valor = None
    
    if match:
        valor_str = match.group(1).strip()
        multiplicador_str = match.group(2)
        limpo = valor_str
        
        # Lógica para limpar e padronizar o número (ex: '50.000' -> 50000.0)
        if ',' in limpo:
            limpo = limpo.replace('.', '').replace(',', '.') 
        else:
            partes = limpo.split('.')
            if len(partes) > 1 and len(partes[-1]) <= 2:
                limpo = "".join(partes[:-1]) + "." + partes[-1]
            else:
                limpo = limpo.replace('.', '')

        limpo = re.sub(r'[^\d\.]', '', limpo)
        
        try:
            valor = float(limpo)
            if multiplicador_str and valor < 1000:
                valor *= 1000
        except ValueError:
            valor = None

    # Lógica de Classificação (Regras)
    if valor is None or valor < 0:
        return valor, 'NÃO SE APLICA'
    
    LIMITE_BAIXO = 1000.00
    LIMITE_MEDIO = 5000.00

    if valor < LIMITE_BAIXO:
        return valor, 'Regra: Aporte Baixo'
    elif LIMITE_BAIXO <= valor <= LIMITE_MEDIO:
        return valor, 'Regra: Aporte Médio'
    elif valor > LIMITE_MEDIO:
        return valor, 'Regra: Aporte Alto'
    else:
        return valor, 'NÃO SE APLICA'

def classificar_texto_fuzzy(texto, mapa_dicionario, threshold=80):
    """
    Classificação Textual: Usa Fuzzy Matching (similaridade de texto) para 
    mapear o input a uma categoria, com retorno padrão unificado para 'NÃO SE APLICA'.
    """
    if pd.isna(texto) or not texto or len(str(texto).strip()) < 2:
        return 'NÃO SE APLICA'
    
    texto_lower = str(texto).lower().strip()
    melhor_categoria = 'NÃO SE APLICA' 
    melhor_score = 0
    
    for palavra_chave, categoria in mapa_dicionario.items():
        
        # Força o match exato para metadados (ex: 'possiveisResgates | false')
        if palavra_chave.startswith(("possiveisresgates", "objetivo")):
            if palavra_chave in texto_lower:
                 return categoria
            else:
                continue 

        # Usa Fuzzy Matching para frases de texto livre
        score = fuzz.partial_ratio(palavra_chave, texto_lower) 
        
        if score > melhor_score:
            melhor_score = score
            melhor_categoria = categoria

    # Só aceita a classificação se o score for alto o suficiente
    if melhor_score >= threshold:
        return melhor_categoria
    else:
        return 'NÃO SE APLICA'
    # ==============================================================================
# 3. EXECUÇÃO DO PIPELINE DE CLASSIFICAÇÃO COM HIERARQUIA
# ==============================================================================

try:
    # 3.1. Leitura dos dados do Excel
    df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME)
    print(f"\nDados lidos da aba '{SHEET_NAME}'. Iniciando classificação...")
        
except Exception as e:
    print(f"\nERRO ao carregar o Excel. Usando dados de SIMULAÇÃO para demonstração.")
    # Estrutura de Simulação (caso o Excel não seja encontrado)
    data = {COLUNA_ORIGEM: ['objetivo | Acumular patrimonio', 'R$ 8.000', '150', 'possiveisResgates | false']}
    df = pd.DataFrame(data)

# ------------------------------------------------------------------------------
# PASSO 1: CLASSIFICAÇÃO DE VALOR DO APORTE (PRIORIDADE ALTA, INDEPENDENTE)
# ------------------------------------------------------------------------------
# Cria as colunas 'Valor_Numerico' e 'Classificacao_Aporte'
df[['Valor_Numerico', 'Classificacao_Aporte']] = df[COLUNA_ORIGEM].apply(
    lambda x: pd.Series(classificar_valor_aporte(x))
)
# ------------------------------------------------------------------------------
# PASSO 2: CLASSIFICAÇÃO DE OBJETIVO DO CLIENTE (PRIORIDADE MÉDIA)
# ------------------------------------------------------------------------------
# Remove o prefixo 'objetivo | ' para que a classificação seja mais precisa
df['Texto_Limpo_Obj'] = (
    df[COLUNA_ORIGEM]
    .astype(str)
    .str.replace(r'^objetivo\s*\|\s*', '', regex=True)
    .str.strip()
)
df['Classificacao_Objetivo'] = df['Texto_Limpo_Obj'].apply(
    lambda x: classificar_texto_fuzzy(x, MAPA_OBJETIVOS, threshold=80)
)
df.drop(columns=['Texto_Limpo_Obj'], inplace=True) # Remove coluna auxiliar de limpeza

# ------------------------------------------------------------------------------
# PASSO 3: CLASSIFICAÇÃO DE RESGATE (PRIORIDADE BAIXA, CONDICIONAL)
# ------------------------------------------------------------------------------
# Regra de Hierarquia: Se for classificado como Objetivo (Passo 2), IGNORE a classificação de Resgate.

# Condição para um Objetivo ser considerado válido (diferente de 'NÃO SE APLICA')
CONDICAO_OBJETIVO_VALIDO = (df['Classificacao_Objetivo'] != 'NÃO SE APLICA')

# Função que aplica o Resgate APENAS se o Objetivo não foi identificado
def aplicar_classificacao_resgate_condicional(row):
    # Verifica a condição de Hierarquia
    if CONDICAO_OBJETIVO_VALIDO[row.name]:
        return 'NÃO SE APLICA'
    # Caso contrário, tenta classificar o Resgate
    else:
        # Usa um threshold (85) mais alto para ser mais seletivo no Resgate
        return classificar_texto_fuzzy(row[COLUNA_ORIGEM], MAPA_RESGATE, threshold=85)

df['Classificacao_Resgate'] = df.apply(aplicar_classificacao_resgate_condicional, axis=1)

# ==============================================================================
# 4. EXPORTAÇÃO DOS RESULTADOS
# ==============================================================================

# Exibe as colunas relevantes para conferência
print("\n--- RESULTADO FINAL DO PIPELINE UNIFICADO (Amostra) ---")
print(df[[COLUNA_ORIGEM, 'Valor_Numerico', 'Classificacao_Aporte', 'Classificacao_Objetivo', 'Classificacao_Resgate']].head(15))

# Salva o resultado em um novo arquivo Excel
OUTPUT_FILE = 'Case - Wpp_PIPELINE_CLASSIFICADO_final.xlsx'
df.to_excel(OUTPUT_FILE, index=False)
print(f"\nSucesso! Arquivo '{OUTPUT_FILE}' gerado com as três novas colunas de classificação.")


Dicionário 'dicionario_objetivo.json' carregado com sucesso.
Dicionário 'dicionario_resgate.json' carregado com sucesso.

ERRO ao carregar o Excel. Usando dados de SIMULAÇÃO para demonstração.

--- RESULTADO FINAL DO PIPELINE UNIFICADO (Amostra) ---
                            input  Valor_Numerico Classificacao_Aporte  \
0  objetivo | Acumular patrimonio             NaN        NÃO SE APLICA   
1                        R$ 8.000          8000.0   Regra: Aporte Alto   
2                             150           150.0  Regra: Aporte Baixo   
3       possiveisResgates | false             NaN        NÃO SE APLICA   

                Classificacao_Objetivo  Classificacao_Resgate  
0  ACÚMULO DE PATRIMÔNIO / CRESCIMENTO          NÃO SE APLICA  
1                        NÃO SE APLICA          NÃO SE APLICA  
2                        NÃO SE APLICA          NÃO SE APLICA  
3                        NÃO SE APLICA  NÃO PRETENDE RESGATAR  

Sucesso! Arquivo 'Case - Wpp_PIPELINE_CLASSIFICADO_final.x